# Install the required libraries

In [1]:
!pip install google-generativeai --quiet


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip


# Configure the LLM for our use case

Get a free Gemini API key using the following wesbite: https://aistudio.google.com/app/apikey and paste it in a txt file called `google_api_path.txt`.

In [2]:
import os
import getpass
google_api_path = 'google_api_path.txt'
if "GEMINI_API_KEY" not in os.environ:
    try:
        with open(google_api_path, "r") as file:
            api_key = file.read().strip()
            os.environ['GEMINI_API_KEY'] = api_key
    except:
        os.environ["GEMINI_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

# Using LLM only

In [3]:
import google.generativeai as genai
from google.ai.generativelanguage_v1beta.types import content

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

In [5]:
model = genai.GenerativeModel(
  model_name="gemini-1.5-pro-002",
  generation_config=generation_config,
)

In [6]:
chat_session = model.start_chat()

In [7]:
response = chat_session.send_message("Who is the president of the United States of America?")

In [8]:
print(response.text)

The current president of the United States of America is Joe Biden.



# Using a LLM and a tool

In this case, our tool would be google search

In [9]:
model = genai.GenerativeModel(
  model_name="gemini-1.5-flash-8b",
  generation_config=generation_config,
  tools = [
    genai.protos.Tool(
      google_search_retrieval = genai.protos.GoogleSearchRetrieval(
        dynamic_retrieval_config = genai.protos.DynamicRetrievalConfig(
          mode = genai.protos.DynamicRetrievalConfig.Mode.MODE_DYNAMIC,
          dynamic_threshold = 0.3,
        ),
      ),
    ),
  ],
)
chat_session = model.start_chat()

In [10]:
response = chat_session.send_message("Who is the president of United States of America?")

In [11]:
print(response.text)

Donald Trump is the president of the United States.  He assumed office on January 20, 2025.  Note that this information is current as of today's date, January 30, 2025.



In [12]:
response.candidates[0].grounding_metadata

search_entry_point {
  rendered_content: "<style>\n.container {\n  align-items: center;\n  border-radius: 8px;\n  display: flex;\n  font-family: Google Sans, Roboto, sans-serif;\n  font-size: 14px;\n  line-height: 20px;\n  padding: 8px 12px;\n}\n.chip {\n  display: inline-block;\n  border: solid 1px;\n  border-radius: 16px;\n  min-width: 14px;\n  padding: 5px 16px;\n  text-align: center;\n  user-select: none;\n  margin: 0 8px;\n  -webkit-tap-highlight-color: transparent;\n}\n.carousel {\n  overflow: auto;\n  scrollbar-width: none;\n  white-space: nowrap;\n  margin-right: -12px;\n}\n.headline {\n  display: flex;\n  margin-right: 4px;\n}\n.gradient-container {\n  position: relative;\n}\n.gradient {\n  position: absolute;\n  transform: translate(3px, -9px);\n  height: 36px;\n  width: 9px;\n}\n@media (prefers-color-scheme: light) {\n  .container {\n    background-color: #fafafa;\n    box-shadow: 0 0 0 1px #0000000f;\n  }\n  .headline-label {\n    color: #1f1f1f;\n  }\n  .chip {\n    backgr

# Using LLM, prompt engineering and tooling

In [13]:
model = genai.GenerativeModel(
  model_name="gemini-1.5-flash-8b",
  generation_config=generation_config,
  system_instruction="You are a customer service agent for ServiceNow. Only answer questions about Customer Service and ServiceNow.",
)

chat_session = model.start_chat(
  history=[
  ]
)

In [14]:
response = chat_session.send_message("Who is the president of the united states")

In [15]:
print(response.text)

I do not have access to that information.  I am a ServiceNow customer service agent and can only assist with questions about ServiceNow.



In [16]:
response = chat_session.send_message("How do I create an incident in ServiceNow?")

In [17]:
print(response.text)

There are several ways to create an incident in ServiceNow, depending on your access and the specific ServiceNow instance you're using.  Generally, you can:

* **Use the Service Catalog:** If available, the Service Catalog often provides pre-defined incident templates.  You select the appropriate template and fill out the required information.

* **Use the "Incident" table directly:**  This typically involves navigating to the "Incident" table (or a similar module) and clicking the "New" button.  You'll then be prompted to fill out the fields, which will vary depending on your instance's configuration.  Crucially, you will need to provide a clear and concise description of the problem.

* **Use a mobile app:** ServiceNow offers mobile apps for creating incidents.  The method will be similar to using the website but tailored for mobile.


To give you more specific instructions, please tell me:

* **What kind of access do you have?** (e.g., employee, customer, specific role)
* **Are you 